#####Libraries
***

In [1]:
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 11.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
--2022-09-19 12:37:20--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-19 12:37:21--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownl

#####Load Dataset
***

In [2]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
  #άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-05-29','2022-09-25')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-05-29','2022-09-25')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [3]:
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,106.605804,81.597748,158.413406,136.604294,136.802231,256.874237,160.932495,203.371292,218.499496,161.456635,...,102.610245,259.269989,252.285995,33.223026,120.749817,398.641083,339.450012,224.728775,169.534683,56.981373
2021-06-02,105.836273,82.125137,156.835663,135.604462,137.594070,257.275818,161.699493,205.369476,215.377548,161.811569,...,102.156342,262.170013,249.654388,33.380478,118.805565,397.788391,338.470001,227.733368,171.020279,57.433754
2021-06-03,106.605804,82.422371,159.030792,135.212372,134.990860,258.382538,159.350494,205.707352,215.249893,162.205948,...,102.345467,257.790009,249.567001,32.563683,117.322823,399.856415,336.119995,226.196365,170.178452,57.659954
2021-06-04,106.748329,82.489479,159.834412,136.084763,138.425507,259.939758,160.311005,204.404175,216.300400,162.679199,...,102.544052,263.040009,251.868454,32.662086,117.313011,397.582550,332.070007,228.209335,170.990555,57.914417
2021-06-07,107.365860,82.144295,159.295410,135.271179,137.554474,259.401062,159.900497,204.780640,212.471527,163.586243,...,102.194168,260.600006,250.334152,32.278294,117.347450,392.299591,316.019989,229.379440,166.187195,57.537430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,142.240005,105.180000,163.490005,100.773888,96.300003,289.239990,136.449997,214.220001,192.050003,162.449997,...,99.760002,97.650002,282.829987,36.259998,94.980003,531.250000,313.399994,206.630005,192.509048,97.610001
2022-09-13,138.529999,102.199997,158.490005,95.476807,90.389999,277.720001,126.820000,207.850006,184.669998,155.350006,...,95.790001,95.010002,279.480011,35.930000,89.860001,513.960022,294.670013,199.669998,185.940002,95.330002
2022-09-14,139.550003,103.320000,155.919998,94.739998,90.639999,277.959991,128.550003,205.080002,180.190002,156.300003,...,94.080002,97.660004,280.149994,36.330002,89.459999,509.769989,290.000000,199.410004,185.910004,97.669998


In [4]:
rd1= rd.pct_change()
rd1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,-0.007218,0.006463,-0.009960,-0.007319,0.005788,0.001563,0.004766,0.009825,-0.014288,0.002198,...,-0.004424,0.011185,-0.010431,0.004739,-0.016101,-0.002139,-0.002887,0.013370,0.008763,0.007939
2021-06-03,0.007271,0.003619,0.013996,-0.002891,-0.018919,0.004302,-0.014527,0.001645,-0.000593,0.002437,...,0.001851,-0.016707,-0.000350,-0.024469,-0.012480,0.005199,-0.006943,-0.006749,-0.004922,0.003938
2021-06-04,0.001337,0.000814,0.005053,0.006452,0.025444,0.006027,0.006028,-0.006335,0.004880,0.002918,...,0.001940,0.020365,0.009222,0.003022,-0.000084,-0.005687,-0.012049,0.008899,0.004772,0.004413
2021-06-07,0.005785,-0.004185,-0.003372,-0.005979,-0.006292,-0.002072,-0.002561,0.001842,-0.017702,0.005576,...,-0.003412,-0.009276,-0.006092,-0.011750,0.000294,-0.013288,-0.048333,0.005127,-0.028091,-0.006509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,0.005798,0.004489,-0.001283,0.015859,-0.002176,0.009176,0.023861,-0.001119,0.005234,0.025309,...,0.012998,0.014756,0.000495,0.003876,0.015720,0.013178,0.007264,0.006969,0.004114,0.011503
2022-09-13,-0.026083,-0.028332,-0.030583,-0.052564,-0.061371,-0.039828,-0.070575,-0.029736,-0.038428,-0.043706,...,-0.039796,-0.027035,-0.011844,-0.009101,-0.053906,-0.032546,-0.059764,-0.033683,-0.034123,-0.023358
2022-09-14,0.007363,0.010959,-0.016216,-0.007717,0.002766,0.000864,0.013641,-0.013327,-0.024259,0.006115,...,-0.017852,0.027892,0.002397,0.011133,-0.004451,-0.008152,-0.015848,-0.001302,-0.000161,0.024546


In [5]:
#prepare the ranks for next month (22 working days later)
rd1_next_month = rd1.shift(-22)
rd1_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,0.014382,0.007921,0.004290,0.006533,-0.031250,0.008478,-0.002090,0.008721,0.000143,0.012985,...,0.008100,-0.008508,0.019087,0.011988,0.007984,0.011113,0.007335,0.005688,0.011905,0.002853
2021-06-02,0.007964,-0.000704,-0.002232,-0.001212,0.001522,-0.000279,0.022724,0.003895,-0.001760,0.009345,...,-0.001646,0.004291,-0.017405,0.001445,0.010081,0.011040,-0.005539,0.014799,0.003959,-0.001423
2021-06-03,0.004862,0.000470,0.001981,-0.010569,-0.009989,-0.014865,0.046927,0.010505,-0.016103,0.006291,...,-0.028993,0.008269,-0.015732,0.008367,0.002139,0.002199,-0.025472,0.004065,0.000620,-0.028495
2021-06-04,0.008814,0.004458,0.001020,0.010537,-0.013964,0.007039,0.005670,0.010208,0.014478,0.008375,...,0.002596,0.015343,0.010468,0.018026,-0.019206,0.003413,0.004367,0.001669,0.025224,-0.015643
2021-06-07,-0.003426,-0.003621,-0.016691,-0.017855,-0.017350,-0.030531,0.009422,0.007139,-0.009689,-0.020353,...,-0.032371,-0.007000,-0.031835,-0.016020,-0.019502,-0.004154,-0.021005,-0.014125,-0.004833,-0.004469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#first: ranks assigned in order they appear in the array
rd2=rd1_next_month.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,39,22,16,18,1,26,7,29,9,38,...,24,2,42,36,23,33,21,17,35,13
2021-06-02,35,14,6,12,19,16,45,23,8,37,...,9,25,1,18,39,40,3,42,24,10
2021-06-03,37,28,32,15,16,12,45,43,9,39,...,4,41,10,42,33,34,6,36,30,5
2021-06-04,28,18,9,33,5,22,20,30,38,27,...,13,39,32,43,3,16,17,11,45,4
2021-06-07,35,34,12,9,10,3,44,43,22,6,...,1,27,2,13,8,32,5,16,30,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-10,22,19,5,39,4,27,41,7,20,42,...,34,37,10,17,38,35,24,23,18,30
2022-08-11,35,30,27,8,4,13,3,29,17,11,...,14,33,44,45,7,23,5,21,20,37
2022-08-12,34,36,7,17,29,26,40,10,4,32,...,5,44,28,37,20,15,8,24,25,43


In [7]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-01,5,3,2,2,1,3,1,4,1,5,...,3,1,5,4,3,4,3,2,4,2
2021-06-02,4,2,1,2,3,2,5,3,1,5,...,1,3,1,2,5,5,1,5,3,2
2021-06-03,5,4,4,2,2,2,5,5,1,5,...,1,5,2,5,4,4,1,4,4,1
2021-06-04,4,2,1,4,1,3,3,4,5,3,...,2,5,4,5,1,2,2,2,5,1
2021-06-07,4,4,2,1,2,1,5,5,3,1,...,1,3,1,2,1,4,1,2,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-10,3,3,1,5,1,3,5,1,3,5,...,4,5,2,2,5,4,3,3,2,4
2022-08-11,4,4,3,1,1,2,1,4,2,2,...,2,4,5,5,1,3,1,3,3,5
2022-08-12,4,4,1,2,4,3,5,2,1,4,...,1,5,4,5,3,2,1,3,3,5


In [8]:
#for absolute rank feature to use 
rd2_feature=rd1.rank(axis=1,method='first')
rd2_feature=rd2_feature.dropna()
rd2_feature.astype(int)
jk_feature=rd2_feature.copy()
jk_feature.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-02,10,32,5,9,31,24,30,38,3,25,...,12,39,4,29,2,15,14,41,36,35
2021-06-03,37,33,41,15,2,35,4,24,19,30,...,27,3,20,1,5,36,9,11,13,34
2021-06-04,15,13,29,34,45,30,31,4,28,20,...,17,44,36,21,10,5,2,35,27,26
2021-06-07,38,20,23,17,15,28,26,32,3,37,...,22,12,16,8,31,7,1,36,2,14
2021-06-08,8,1,28,32,2,13,44,9,33,23,...,19,36,27,35,34,24,45,22,38,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,22,19,5,39,4,27,41,7,20,42,...,34,37,10,17,38,35,24,23,18,30
2022-09-13,35,30,27,8,4,13,3,29,17,11,...,14,33,44,45,7,23,5,21,20,37
2022-09-14,34,36,7,17,29,26,40,10,4,32,...,5,44,28,37,20,15,8,24,25,43


In [9]:
r5_feature=rd2_feature.copy()
rd3_feature=rank_data(r5_feature)
rd3_feature

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-02,2,4,1,1,4,3,4,5,1,3,...,2,5,1,4,1,2,2,5,4,4
2021-06-03,5,4,5,2,1,4,1,3,3,4,...,3,1,3,1,1,4,1,2,2,4
2021-06-04,2,2,4,4,5,4,4,1,4,3,...,2,5,4,3,2,1,1,4,3,3
2021-06-07,5,3,3,2,2,4,3,4,1,5,...,3,2,2,1,4,1,1,4,1,2
2021-06-08,1,1,4,4,1,2,5,1,4,3,...,3,4,3,4,4,3,5,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-12,3,3,1,5,1,3,5,1,3,5,...,4,5,2,2,5,4,3,3,2,4
2022-09-13,4,4,3,1,1,2,1,4,2,2,...,2,4,5,5,1,3,1,3,3,5
2022-09-14,4,4,1,2,4,3,5,2,1,4,...,1,5,4,5,3,2,1,3,3,5


In [11]:
rd3.loc[rd3.index=='2022-08-19']

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,


#####Functions
***

In [12]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    # max_date = np.max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [13]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [14]:
def predictions(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      # print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-24] #-23 , -20 ,-2
      
      Y_train= y.iloc[:-24] #-23 , -20

      print(Y_train.tail(2))
      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      flag=True
      for i in range(23,0,-1): #from 4/01 and later -->predictions
        if(flag):
          first=X.iloc[-(i+1):-i].index
          print('this is the first day of predictions', first)
          flag=False
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        # print(Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      X_validation=X.tail(1) 
      last=X_validation.index
      print('this is last for prediction',last)
      # print(X_validation)
      Y_validation= y.tail(1)
      print(Y_validation.index) 
      probability= model.predict_proba(X_validation)
      print(probability[0])
      tmp.append(probability[0])
      prob[j]=tmp
  return prob , first , last 

In [15]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob : ',prob)
      mask1 = (rd.loc[(rd.index.date>=prob_final['date'][k+1])].index)
      prob1_table = rd.loc[mask1]
      prob1 = prob1_table.shift(-21).head(1)
      print('prob1 : ',prob1)

      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob teleutaio :', prob)
      prob1 = pd.DataFrame(prob_table.iloc[1]).T
      print('prob1 : ',prob1)
      
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [16]:
def create_RPS(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

####**For 29 May-24 June** 

#####Features
***

In [17]:
stock=dict()
for i in assets:
  stock[i]=pdr.get_data_yahoo(i,'2021-05-29','2022-06-25') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-05-27') & (stock['ABBV'].index <= '2022-06-24')]))
# stock['ABBV']

18


In [18]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-05-27']['Rank'])

Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    5.0
Name: Rank, dtype: float64
Date
2022-05-27    3.0
Name: Rank, dtype: float64
Date
2022-05-27    2.0
Name: Rank, dtype: float64
Date
2022-05-27    3.0
Name: Rank, dtype: float64
Date
2022-05-27    2.0
Name: Rank, dtype: float64
Date
2022-05-27    1.0
Name: Rank, dtype: float64
Date
2022-05-27    5.0
Name: Rank, dtype: float64
Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    1.0
Name: Rank, dtype: float64
Date
2022-05-27    2.0
Name: Rank, dtype: float64
Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    5.0
Name: Rank, dtype: float64
Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    2.0
Name: Rank, dtype: float64
Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    3.0
Name: Rank, dtype: float64
Date
2022-05-27    5.0
Name: Rank, dtype: float64
Date
2022-05-27    4.0
Name: Rank, dtype: float64
Date
2022-05-27    5.0
Name: Rank, dtype: float64


#####Gradient Boosting (baseline)
***

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-05-18    5
2022-05-19    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.11924118 0.26254775 0.06816564 0.42355635 0.12648907]
Date
2022-05-18    3
2022-05-19    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.59291426 0.00999005 0.08649642 0.01041166 0.30018761]
Date
2022-05-18    2
2022-05-19    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-23  148.065018  91.689781  168.615479  96.392845  93.042061   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-23  238.452896  112.440002  194.904602  154.861267  141.121872  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-23  91.416611  73.809998  270.594604  34.059608  73.720894   

                   UNH         URI           V       VRSK        XOM  
Date                                                                  
2022-05-23  498.246613  235.839996  196.292328  170.68544  84.390045  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-24  150.928238  92.870499  172.150925  99.170822  97.111656   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-24  248.137924  116.459999  193.575409  164.681396  145.605087  ...   

                  PRU   PYPL         RE        ROL       ROST         UNH  \
Date                                                                        
2022-05-24  96.279518  77.68  281.48877  35.017067  77.258453  494.089691   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-24  248.289993  205.146637  172.702087  86.063782  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                      

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-06-23  150.220001  95.05336  171.254623  102.566132  100.742371   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-23  244.902954  121.139999  197.539993  174.073257  153.789993  ...   

                  PRU       PYPL        RE        ROL       ROST         UNH  \
Date                                                                           
2022-06-23  95.745781  81.650002  272.1651  36.393417  82.340591  527.813843   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-23  277.829987  213.891159  182.465805  89.114151  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-24  150.869995  95.668518  171.234695  102.247513  98.657707   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-24  240.463562  114.809998  197.539993  171.065475  149.830002  ...   

                  PRU       PYPL          RE       ROL       ROST         UNH  \
Date                                                                            
2022-06-24  94.935303  77.040001  270.187073  36.12413  77.686951  529.927246   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-24  276.209991  212.114319  182.705399  88.767509  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
20

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.147373,0.221914,0.403244,0.111424,0.121026,0.106150,0.309915,0.301448,0.372133,0.450812,...,0.086547,0.274793,0.216482,0.081133,0.136599,0.183677,0.086258,0.100516,0.323773,0.279620
2022-05-23,0.128453,0.199860,0.163503,0.107068,0.408759,0.183001,0.482241,0.367372,0.536069,0.119747,...,0.563563,0.447709,0.099764,0.153546,0.001095,0.411085,0.575053,0.166532,0.500162,0.221376
2022-05-24,0.165077,0.198485,0.177989,0.414169,0.117249,0.281442,0.033152,0.356246,0.118122,0.112576,...,0.103614,0.011804,0.101088,0.013443,0.626251,0.013261,0.435191,0.277905,0.019957,0.254773
2022-05-25,0.146034,0.213748,0.203031,0.093729,0.131859,0.233031,0.268446,0.151101,0.047551,0.196623,...,0.481375,0.071123,0.167742,0.270333,0.154846,0.082510,0.234423,0.064082,0.547344,0.363182
2022-05-26,0.079063,0.152816,0.544876,0.121026,0.071586,0.130938,0.008817,0.139786,0.160219,0.085174,...,0.232902,0.277453,0.195320,0.186652,0.203553,0.027093,0.179542,0.195596,0.209305,0.151754
2022-05-27,0.201588,0.264622,0.053849,0.066810,0.002370,0.152334,0.154941,0.022094,0.092451,0.264016,...,0.085372,0.167120,0.382290,0.024757,0.225741,0.077904,0.556862,0.133740,0.256319,0.090346
2022-05-31,0.160971,0.010248,0.058047,0.128204,0.396770,0.273845,0.662317,0.113638,0.102824,0.034259,...,0.098419,0.179048,0.074919,0.237250,0.310133,0.104944,0.133453,0.118615,0.051091,0.512003
2022-06-01,0.163106,0.303912,0.066177,0.118150,0.100423,0.095718,0.304376,0.203063,0.022730,0.066224,...,0.075255,0.379727,0.026411,0.083535,0.011284,0.258805,0.123165,0.170279,0.174880,0.048093
2022-06-02,0.220971,0.130636,0.297651,0.088508,0.134592,0.107222,0.344459,0.170424,0.389762,0.175404,...,0.093495,0.023342,0.192044,0.310358,0.021045,0.419697,0.642097,0.165196,0.216450,0.140322


In [20]:
print(RPS1.mean(axis=1).mean())

0.20568928391984298


In [21]:
print(RPS1.mean(axis=1))

date
2022-05-20    0.176319
2022-05-23    0.253994
2022-05-24    0.170486
2022-05-25    0.193759
2022-05-26    0.182190
2022-05-27    0.201731
2022-05-31    0.194443
2022-06-01    0.193768
2022-06-02    0.182612
2022-06-03    0.212533
2022-06-06    0.242161
2022-06-07    0.178189
2022-06-08    0.244614
2022-06-09    0.171075
2022-06-10    0.207151
2022-06-13    0.252371
2022-06-14    0.233069
2022-06-15    0.188798
2022-06-16    0.227134
2022-06-17    0.240242
2022-06-21    0.192699
2022-06-22    0.197241
2022-06-23    0.185461
2022-06-24    0.214502
dtype: float64


In [22]:
#save results 
RPS1.to_excel('29_May_24_June-grad_boost(baseline)-real_data.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('29_May_24_June-PROBABILITIES-grad_boost(baseline)-real_data.xlsx')

#####After feature importances and tuning
***

In [23]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-05-18    5
2022-05-19    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.14832897 0.19567264 0.17534646 0.21252561 0.26812632]
Date
2022-05-18    3
2022-05-19    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-06-24'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16670054 0.22510734 0.16607855 0.19913291 0.24298065]
Date
2022-05-18    2
2022-05-19    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-05-20'], dtype='datetime64[ns]', name='Date'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-23  148.065018  91.689781  168.615479  96.392845  93.042061   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-23  238.452896  112.440002  194.904602  154.861267  141.121872  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-23  91.416611  73.809998  270.594604  34.059608  73.720894   

                   UNH         URI           V       VRSK        XOM  
Date                                                                  
2022-05-23  498.246613  235.839996  196.292328  170.68544  84.390045  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date         

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-24  150.928238  92.870499  172.150925  99.170822  97.111656   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-24  248.137924  116.459999  193.575409  164.681396  145.605087  ...   

                  PRU   PYPL         RE        ROL       ROST         UNH  \
Date                                                                        
2022-05-24  96.279518  77.68  281.48877  35.017067  77.258453  494.089691   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-24  248.289993  205.146637  172.702087  86.063782  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
Date                      

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-06-23  150.220001  95.05336  171.254623  102.566132  100.742371   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-23  244.902954  121.139999  197.539993  174.073257  153.789993  ...   

                  PRU       PYPL        RE        ROL       ROST         UNH  \
Date                                                                           
2022-06-23  95.745781  81.650002  272.1651  36.393417  82.340591  527.813843   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-23  277.829987  213.891159  182.465805  89.114151  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-24  150.869995  95.668518  171.234695  102.247513  98.657707   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-06-24  240.463562  114.809998  197.539993  171.065475  149.830002  ...   

                  PRU       PYPL          RE       ROL       ROST         UNH  \
Date                                                                            
2022-06-24  94.935303  77.040001  270.187073  36.12413  77.686951  529.927246   

                   URI           V        VRSK        XOM  
Date                                                       
2022-06-24  276.209991  212.114319  182.705399  88.767509  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ      ALLE       AMAT  \
20

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20,0.086219,0.236855,0.281249,0.079907,0.134705,0.132752,0.256995,0.247344,0.102402,0.239139,...,0.112007,0.139312,0.129254,0.226824,0.128387,0.098266,0.201612,0.074076,0.249446,0.275797
2022-05-23,0.145553,0.246400,0.121365,0.077110,0.147738,0.102961,0.149855,0.241738,0.287368,0.079227,...,0.260600,0.291700,0.114836,0.083702,0.128387,0.287200,0.247153,0.117635,0.265310,0.164540
2022-05-24,0.109619,0.239369,0.059963,0.111078,0.121961,0.132752,0.202964,0.247461,0.128172,0.239139,...,0.112007,0.139393,0.069944,0.083702,0.216929,0.215671,0.242945,0.242209,0.105432,0.166736
2022-05-25,0.098452,0.239897,0.249284,0.133066,0.134705,0.069930,0.202157,0.115889,0.101908,0.079227,...,0.260600,0.139548,0.116436,0.130352,0.216929,0.215671,0.116147,0.119522,0.265310,0.177044
2022-05-26,0.096391,0.119492,0.119384,0.126511,0.204186,0.266178,0.256995,0.076062,0.128172,0.232997,...,0.112007,0.139112,0.127934,0.226824,0.113037,0.215671,0.201612,0.120238,0.260262,0.286209
2022-05-27,0.089532,0.242047,0.247562,0.076240,0.139016,0.132752,0.203031,0.076224,0.101908,0.129256,...,0.066156,0.139776,0.220922,0.115567,0.221885,0.068824,0.247153,0.117939,0.263896,0.275797
2022-05-31,0.157326,0.118028,0.056773,0.128751,0.220268,0.266178,0.282719,0.122712,0.064572,0.079218,...,0.126833,0.308605,0.069729,0.244908,0.130783,0.145565,0.119124,0.070564,0.056059,0.177044
2022-06-01,0.096904,0.242769,0.121365,0.127784,0.119143,0.132752,0.279380,0.127742,0.064572,0.079218,...,0.244949,0.307600,0.274106,0.083702,0.251555,0.288628,0.143602,0.120238,0.123003,0.275797
2022-06-02,0.282817,0.239897,0.109014,0.076476,0.147738,0.132752,0.172347,0.122614,0.287993,0.232997,...,0.116716,0.139776,0.217206,0.226824,0.120546,0.218110,0.199418,0.070564,0.260262,0.275797


In [25]:
print(RPS2.mean(axis=1).mean())

0.1594174925638895


In [26]:
print(RPS2.mean(axis=1))

date
2022-05-20    0.159974
2022-05-23    0.171017
2022-05-24    0.149919
2022-05-25    0.152674
2022-05-26    0.157342
2022-05-27    0.156221
2022-05-31    0.160023
2022-06-01    0.169708
2022-06-02    0.155712
2022-06-03    0.161975
2022-06-06    0.151354
2022-06-07    0.150281
2022-06-08    0.163662
2022-06-09    0.157703
2022-06-10    0.161588
2022-06-13    0.167207
2022-06-14    0.162148
2022-06-15    0.166080
2022-06-16    0.166346
2022-06-17    0.169926
2022-06-21    0.149465
2022-06-22    0.152400
2022-06-23    0.150868
2022-06-24    0.162428
dtype: float64


In [27]:

#save results 
RPS2.to_excel('29_May_24_June-grad_boost(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('29_May_24_June-PROBABILITIES-grad_boost(with_feature_imp)-real_data.xlsx')

####**For 26 June.-22 July.**

#####Features
***

In [28]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-06-26','2022-07-23') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-06-28  113.260002  114.110001  112.839996  113.000000  107.356354
 2021-06-29  112.650002  112.750000  111.610001  112.300003  106.691330
 2021-06-30  112.209999  112.919998  111.629997  112.639999  107.014336
 2021-07-01  112.949997  114.550003  112.889999  114.260002  108.553436
 2021-07-02  114.519997  115.489998  113.889999  115.169998  109.417984
 ...                ...         ...         ...         ...         ...
 2022-07-18  153.800003  154.470001  148.889999  149.570007  149.570007
 2022-07-19  150.000000  150.229996  148.800003  149.740005  149.740005
 2022-07-20  149.529999  150.449997  147.339996  147.690002  147.690002
 2022-07-21  147.240005  147.839996  146.020004  147.750000  147.750000
 2022-07-22  148.470001  149.289993  147.580002  148.470001  148.470001
 
 [270 rows x 5 columns],
 'AEP':                  Open

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-06-26') & (stock['ABBV'].index <= '2022-07-22')]))

19


In [29]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-06-24']['Rank'])

Date
2022-06-24    4.0
Name: Rank, dtype: float64
Date
2022-06-24    5.0
Name: Rank, dtype: float64
Date
2022-06-24    5.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    3.0
Name: Rank, dtype: float64
Date
2022-06-24    1.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    4.0
Name: Rank, dtype: float64
Date
2022-06-24    3.0
Name: Rank, dtype: float64
Date
2022-06-24    3.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    1.0
Name: Rank, dtype: float64
Date
2022-06-24    2.0
Name: Rank, dtype: float64
Date
2022-06-24    3.0
Name: Rank, dtype: float64
Date
2022-06-24    1.0
Name: Rank, dtype: float64
Date
2022-06-24    4.0
Name: Rank, dtype: float64
Date
2022-06-24    5.0
Name: Rank, dtype: float64
Date
2022-06-24    5.0
Name: Rank, dtype: float64


#####Gradient Boosting (baseline)
***

In [30]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-06-15    1
2022-06-16    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.53951615 0.00871255 0.40478065 0.04062306 0.0063676 ]
Date
2022-06-15    1
2022-06-16    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.03692112 0.09051586 0.68380342 0.04870832 0.14005128]
Date
2022-06-15    2
2022-06-16    3
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                ABBV        AEP         AIZ        ALLE        AMAT         AMP  \
Date                                                                            
2022-06-21  147.75  92.066818  168.087662  103.014191  103.575127  244.206192   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-06-21  124.629997  196.050003  175.318207  150.179993  ...  94.955063   

             PYPL          RE       ROL       ROST         UNH         URI  \
Date                                                                         
2022-06-21  82.75  268.616547  36.46323  83.317162  520.825806  279.790009   

                    V        VRSK        XOM  
Date                                          
2022-06-21  215.80777  184.402573  86.905602  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-22  148.470001  93.614655  169.043716  102.944489  101.41066   

                   AMP        AMZN     AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-06-22  243.927475  122.419998  198.25  174.591156  153.009995  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-06-22  94.401566  81.050003  270.306335  36.373466  82.948456   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-06-22  519.779053  274.290009  213.322159  184.472458  86.24205  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                 

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-21  141.850006  104.940002  171.145065  105.991302  104.363106   

                  AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                  ...   
2022-07-21  284.73999  138.229996  214.979996  196.25322  162.240005  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-07-21  102.199997  96.559998  279.808899  36.540001  90.691246   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-21  546.604919  316.690002  212.820007  200.236176  94.080002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
D

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-22  140.339996  103.410004  164.163818  102.695572  100.552856   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-07-22  276.98999  133.220001  210.330002  193.584045  157.699997  ...   

                  PRU       PYPL         RE        ROL       ROST         UNH  \
Date                                                                            
2022-07-22  99.110001  93.580002  273.01001  36.400002  87.801399  542.866638   

                   URI           V        VRSK        XOM  
Date                                                       
2022-07-22  307.549988  208.610001  197.900055  94.010002  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-06-17,0.172406,0.141052,0.075917,0.375708,0.677297,0.012697,0.094457,0.042187,0.121920,0.107781,...,0.123458,0.006135,0.115001,0.167454,0.133465,0.123522,0.104788,0.080685,0.129193,0.186972
2022-06-21,0.435452,0.567103,0.202072,0.116904,0.086761,0.060604,0.554764,0.299593,0.094496,0.401922,...,0.233195,0.428775,0.164765,0.222064,0.319818,0.075182,0.555376,0.188551,0.003986,0.187075
2022-06-22,0.112224,0.385171,0.475005,0.107950,0.001952,0.141660,0.050311,0.100977,0.116474,0.149865,...,0.200330,0.135481,0.156374,0.167779,0.257643,0.261273,0.138347,0.094118,0.403394,0.187566
2022-06-23,0.205947,0.554458,0.447030,0.124802,0.074835,0.022082,0.653068,0.181722,0.004025,0.208972,...,0.241886,0.069325,0.066988,0.030192,0.265017,0.366782,0.064111,0.032129,0.285231,0.144660
2022-06-24,0.260227,0.195744,0.167221,0.048698,0.720420,0.519924,0.159692,0.244302,0.543073,0.147763,...,0.072049,0.252534,0.505816,0.315003,0.118974,0.144773,0.407289,0.361512,0.195170,0.103612
2022-06-27,0.074523,0.152219,0.148213,0.059627,0.189049,0.153341,0.148558,0.348884,0.199295,0.172964,...,0.000156,0.255180,0.254630,0.334137,0.355559,0.084859,0.471893,0.051073,0.084685,0.185529
2022-06-28,0.348521,0.210712,0.315952,0.302722,0.156534,0.367904,0.255546,0.117314,0.102360,0.136856,...,0.142974,0.304312,0.012822,0.382339,0.636677,0.110743,0.446904,0.114282,0.030918,0.202252
2022-06-29,0.433707,0.310073,0.104706,0.172966,0.555128,0.298117,0.156744,0.105007,0.081814,0.212385,...,0.038074,0.775961,0.361088,0.333097,0.298341,0.414844,0.080833,0.060743,0.369833,0.705484
2022-06-30,0.066285,0.200127,0.025712,0.111114,0.248491,0.088196,0.193241,0.167043,0.143422,0.123819,...,0.176864,0.518607,0.197190,0.191776,0.621718,0.102998,0.249544,0.461536,0.101676,0.154922


In [31]:
print(RPS1.mean(axis=1).mean())

0.20908729264658585


In [32]:
print(RPS1.mean(axis=1))

date
2022-06-17    0.190597
2022-06-21    0.196688
2022-06-22    0.209103
2022-06-23    0.178413
2022-06-24    0.239875
2022-06-27    0.193586
2022-06-28    0.222476
2022-06-29    0.224219
2022-06-30    0.226658
2022-07-01    0.218698
2022-07-05    0.260636
2022-07-06    0.202788
2022-07-07    0.195364
2022-07-08    0.202149
2022-07-11    0.214669
2022-07-12    0.210595
2022-07-13    0.210031
2022-07-14    0.206479
2022-07-15    0.222463
2022-07-18    0.189755
2022-07-19    0.207555
2022-07-20    0.196583
2022-07-21    0.188152
2022-07-22    0.210565
dtype: float64


In [33]:
#save results 
RPS1.to_excel('26_June_22_July-grad_boost(baseline)-real_data.xlsx')
# prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('26_June_22_July-PROBABILITIES-grad_boost(baseline)-real_data.xlsx')

#####After feature importances and tuning
***

In [34]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [35]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-06-15    1
2022-06-16    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15422885 0.18367968 0.20216152 0.20847648 0.25145347]
Date
2022-06-15    1
2022-06-16    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-07-22'], dtype='datetime64[ns]', name='Date', freq=None)
[0.17311109 0.21406692 0.17085588 0.22667156 0.21529456]
Date
2022-06-15    2
2022-06-16    3
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-06-17'], dtype='datetime64[ns]', name='Date'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                ABBV        AEP         AIZ        ALLE        AMAT         AMP  \
Date                                                                            
2022-06-21  147.75  92.066818  168.087662  103.014191  103.575127  244.206192   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-06-21  124.629997  196.050003  175.318207  150.179993  ...  94.955063   

             PYPL          RE       ROL       ROST         UNH         URI  \
Date                                                                         
2022-06-21  82.75  268.616547  36.46323  83.317162  520.825806  279.790009   

                    V        VRSK        XOM  
Date                                          
2022-06-21  215.80777  184.402573  86.905602  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE       AMAT  \
Date                                                                   
2022-06-22  148.470001  93.614655  169.043716  102.944489  101.41066   

                   AMP        AMZN     AVB         AVY         AXP  ...  \
Date                                                                ...   
2022-06-22  243.927475  122.419998  198.25  174.591156  153.009995  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-06-22  94.401566  81.050003  270.306335  36.373466  82.948456   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-06-22  519.779053  274.290009  213.322159  184.472458  86.24205  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                 

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-21  141.850006  104.940002  171.145065  105.991302  104.363106   

                  AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                  ...   
2022-07-21  284.73999  138.229996  214.979996  196.25322  162.240005  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-07-21  102.199997  96.559998  279.808899  36.540001  90.691246   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-21  546.604919  316.690002  212.820007  200.236176  94.080002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
D

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-22  140.339996  103.410004  164.163818  102.695572  100.552856   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-07-22  276.98999  133.220001  210.330002  193.584045  157.699997  ...   

                  PRU       PYPL         RE        ROL       ROST         UNH  \
Date                                                                            
2022-07-22  99.110001  93.580002  273.01001  36.400002  87.801399  542.866638   

                   URI           V        VRSK        XOM  
Date                                                       
2022-07-22  307.549988  208.610001  197.900055  94.010002  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-06-17,0.280692,0.124174,0.127587,0.264930,0.191869,0.113418,0.157799,0.071879,0.116742,0.075341,...,0.112703,0.283937,0.133253,0.084771,0.119485,0.073787,0.127359,0.121792,0.124697,0.275406
2022-06-21,0.098109,0.239059,0.109053,0.075263,0.234854,0.072665,0.195287,0.256035,0.066255,0.264340,...,0.112703,0.144102,0.220543,0.084771,0.096427,0.075374,0.222815,0.115411,0.061863,0.167786
2022-06-22,0.098651,0.230290,0.240513,0.120551,0.124419,0.072665,0.196661,0.116701,0.105886,0.133653,...,0.257072,0.203644,0.111689,0.084771,0.227815,0.203851,0.132102,0.071563,0.243643,0.172346
2022-06-23,0.189972,0.230290,0.109301,0.075263,0.134166,0.126254,0.205021,0.119641,0.104865,0.228735,...,0.112703,0.137035,0.065400,0.128469,0.238250,0.102535,0.108978,0.115411,0.123421,0.104679
2022-06-24,0.149250,0.276819,0.122485,0.072631,0.218163,0.241476,0.246507,0.119641,0.269011,0.073784,...,0.066216,0.291545,0.280242,0.244791,0.119485,0.147045,0.248447,0.241224,0.117140,0.104679
2022-06-27,0.288463,0.090931,0.119229,0.103525,0.146542,0.107739,0.106167,0.256035,0.288152,0.103501,...,0.057598,0.137601,0.281114,0.229228,0.096750,0.075839,0.244420,0.071563,0.121375,0.105857
2022-06-28,0.293142,0.090931,0.107032,0.129911,0.119949,0.241476,0.246507,0.071879,0.108261,0.133653,...,0.124143,0.138761,0.065400,0.243949,0.227815,0.147045,0.248447,0.115411,0.054824,0.172346
2022-06-29,0.288463,0.224061,0.122365,0.071978,0.146542,0.257928,0.156762,0.237956,0.066488,0.103501,...,0.066216,0.293262,0.280242,0.231165,0.220627,0.291906,0.119392,0.068063,0.122342,0.281282
2022-06-30,0.098651,0.084191,0.273324,0.221359,0.134166,0.126254,0.122048,0.237956,0.120368,0.106689,...,0.249338,0.293262,0.132550,0.131146,0.238250,0.099717,0.197592,0.243427,0.121453,0.104679


In [36]:
print(RPS2.mean(axis=1).mean())

0.158906389413325


In [37]:
print(RPS2.mean(axis=1))

date
2022-06-17    0.166986
2022-06-21    0.151724
2022-06-22    0.152002
2022-06-23    0.151115
2022-06-24    0.166333
2022-06-27    0.162510
2022-06-28    0.163522
2022-06-29    0.164616
2022-06-30    0.162430
2022-07-01    0.166949
2022-07-05    0.163525
2022-07-06    0.149617
2022-07-07    0.158118
2022-07-08    0.147596
2022-07-11    0.170389
2022-07-12    0.162621
2022-07-13    0.165623
2022-07-14    0.169054
2022-07-15    0.165489
2022-07-18    0.145667
2022-07-19    0.157555
2022-07-20    0.145281
2022-07-21    0.145823
2022-07-22    0.159209
dtype: float64


In [38]:
#save results 
RPS2.to_excel('26_June_22_July-grad_boost(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('26_June_22_July-PROBABILITIES-grad_boost(with_feature_imp)-real_data.xlsx')

####**For 24 July-19 August**

#####Features
***

In [39]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-07-22','2022-08-20') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-07-22  116.639999  117.910004  116.389999  117.540001  112.914551
 2021-07-23  117.980003  118.669998  117.570000  118.190002  113.538971
 2021-07-26  118.129997  118.470001  117.269997  117.790001  113.154709
 2021-07-27  117.790001  118.300003  116.760002  117.959999  113.318008
 2021-07-28  117.629997  118.930000  117.300003  118.550003  113.884796
 ...                ...         ...         ...         ...         ...
 2022-08-15  143.110001  143.279999  140.419998  142.289993  142.289993
 2022-08-16  142.110001  143.979996  142.080002  142.550003  142.550003
 2022-08-17  141.419998  142.759995  140.550003  141.440002  141.440002
 2022-08-18  141.490005  141.580002  139.820007  141.289993  141.289993
 2022-08-19  141.779999  143.360001  141.460007  141.850006  141.850006
 
 [273 rows x 5 columns],
 'AEP':                   Ope

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-07-24') & (stock['ABBV'].index <= '2022-08-19')]))

20


In [40]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-07-22']['Rank'])

Date
2022-07-22    4.0
Name: Rank, dtype: float64
Date
2022-07-22    4.0
Name: Rank, dtype: float64
Date
2022-07-22    1.0
Name: Rank, dtype: float64
Date
2022-07-22    1.0
Name: Rank, dtype: float64
Date
2022-07-22    1.0
Name: Rank, dtype: float64
Date
2022-07-22    2.0
Name: Rank, dtype: float64
Date
2022-07-22    1.0
Name: Rank, dtype: float64
Date
2022-07-22    3.0
Name: Rank, dtype: float64
Date
2022-07-22    4.0
Name: Rank, dtype: float64
Date
2022-07-22    2.0
Name: Rank, dtype: float64
Date
2022-07-22    4.0
Name: Rank, dtype: float64
Date
2022-07-22    5.0
Name: Rank, dtype: float64
Date
2022-07-22    4.0
Name: Rank, dtype: float64
Date
2022-07-22    5.0
Name: Rank, dtype: float64
Date
2022-07-22    3.0
Name: Rank, dtype: float64
Date
2022-07-22    3.0
Name: Rank, dtype: float64
Date
2022-07-22    1.0
Name: Rank, dtype: float64
Date
2022-07-22    5.0
Name: Rank, dtype: float64
Date
2022-07-22    3.0
Name: Rank, dtype: float64
Date
2022-07-22    5.0
Name: Rank, dtype: float64


#####Gradient Boosting (baseline)
***

In [41]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-07-12    4.0
2022-07-13    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.06954939 0.02904642 0.44729922 0.11387693 0.34022804]
Date
2022-07-12    3.0
2022-07-13    5.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.04099332 0.11446971 0.0674013  0.60548394 0.17165173]
Date
2022-07-12    4.0
2022-07-13    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-15  142.289993  104.559998  166.573883  106.738075  109.499977   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-15  286.339996  143.179993  218.380005  200.366501  165.399994  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-15  103.603516  101.510002  273.487122  36.900002  89.844223   

                   UNH         URI           V        VRSK    XOM  
Date                                                               
2022-07-15  542.936401  337.519989  216.419998  205.327682  92.32  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-18  142.550003  104.459999  169.531708  107.992645  108.213264   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-18  289.059998  144.779999  219.240005  203.294617  165.350006  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-18  103.959343  102.080002  275.634125  37.490002  92.315536   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-18  544.411743  339.309998  217.139999  204.149658  91.459999  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-16  142.509995  100.190002  155.759995  93.150002  88.919998   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-08-16  277.350006  126.279999  201.320007  179.360001  156.130005  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-08-16  92.540001  96.400002  285.019989  35.799999  89.559998   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-08-16  522.909973  288.709991  195.369995  183.979996  94.830002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMA

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-07-14,0.180432,0.156603,0.072957,0.387513,0.739539,0.290809,0.142654,0.001013,0.287580,0.252563,...,0.126806,0.151438,0.046703,0.297691,0.119552,0.015805,0.276993,0.610454,0.278682,0.078312
2022-07-15,0.102346,0.129489,0.399642,0.550584,0.110914,0.193771,0.170042,0.060688,0.332066,0.090336,...,0.121444,0.155650,0.282998,0.217844,0.496829,0.104549,0.129775,0.251126,0.262133,0.003151
2022-07-18,0.085335,0.111571,0.363554,0.248606,0.129638,0.097500,0.500805,0.382391,0.115661,0.322882,...,0.116360,0.050613,0.362949,0.126127,0.251172,0.285087,0.205277,0.190372,0.036791,0.180825
2022-07-19,0.199566,0.461250,0.118787,0.272166,0.053700,0.187774,0.314737,0.641298,0.230438,0.101607,...,0.106498,0.006824,0.369782,0.099792,0.291849,0.098428,0.258955,0.313807,0.251868,0.216719
2022-07-20,0.399813,0.016530,0.055072,0.183432,0.124695,0.211383,0.176993,0.107047,0.145676,0.051061,...,0.177349,0.568262,0.153516,0.114840,0.114451,0.270888,0.302824,0.330724,0.139252,0.128474
2022-07-21,0.206552,0.102291,0.403491,0.241758,0.096201,0.168749,0.133047,0.041161,0.030469,0.232850,...,0.167527,0.146236,0.224648,0.249270,0.189688,0.383068,0.313096,0.130295,0.154578,0.284350
2022-07-22,0.145146,0.014440,0.127224,0.031247,0.522500,0.137552,0.599357,0.295944,0.089964,0.079101,...,0.254046,0.219527,0.093204,0.196563,0.386522,0.037601,0.087805,0.001937,0.395611,0.075334
2022-07-25,0.389804,0.095207,0.116331,0.040306,0.426509,0.256144,0.166255,0.402315,0.123423,0.156694,...,0.029950,0.438832,0.120575,0.268152,0.172861,0.163585,0.194549,0.075318,0.191808,0.128886
2022-07-26,0.078315,0.212834,0.015465,0.100100,0.210952,0.125259,0.417774,0.128696,0.041406,0.096881,...,0.287777,0.122025,0.074614,0.001154,0.149751,0.102563,0.294108,0.062233,0.288608,0.172895


In [42]:
print(RPS1.mean(axis=1).mean())

0.2083128112160323


In [43]:
print(RPS1.mean(axis=1))

date
2022-07-14    0.236049
2022-07-15    0.211825
2022-07-18    0.198755
2022-07-19    0.205871
2022-07-20    0.175642
2022-07-21    0.213260
2022-07-22    0.206939
2022-07-25    0.204835
2022-07-26    0.202686
2022-07-27    0.202567
2022-07-28    0.173269
2022-07-29    0.223490
2022-08-01    0.204654
2022-08-02    0.237174
2022-08-03    0.196462
2022-08-04    0.226137
2022-08-05    0.223985
2022-08-08    0.189224
2022-08-09    0.230507
2022-08-10    0.194157
2022-08-11    0.211964
2022-08-12    0.205352
2022-08-15    0.216928
2022-08-16    0.207777
dtype: float64


In [44]:
#save results 
#save results 
RPS1.to_excel('24_July_19_August-grad_boost(baseline)-real_data.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('24_July_19_August-PROBABILITIES-grad_boost(baseline)-real_data.xlsx')

#####After feature importances and tuning
***

In [45]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [46]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-07-12    4.0
2022-07-13    1.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.17656759 0.20006711 0.15852248 0.20676307 0.25807974]
Date
2022-07-12    3.0
2022-07-13    5.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-08-16'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16069784 0.21157868 0.1819252  0.23177559 0.21402269]
Date
2022-07-12    4.0
2022-07-13    3.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-07-14'], dtype='datetime64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-15  142.289993  104.559998  166.573883  106.738075  109.499977   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-15  286.339996  143.179993  218.380005  200.366501  165.399994  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-15  103.603516  101.510002  273.487122  36.900002  89.844223   

                   UNH         URI           V        VRSK    XOM  
Date                                                               
2022-07-15  542.936401  337.519989  216.419998  205.327682  92.32  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                     
2022-07-18  142.550003  104.459999  169.531708  107.992645  108.213264   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-07-18  289.059998  144.779999  219.240005  203.294617  165.350006  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-07-18  103.959343  102.080002  275.634125  37.490002  92.315536   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-07-18  544.411743  339.309998  217.139999  204.149658  91.459999  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV     AEP         AIZ       ALLE       AMAT         AMP  \
Date                                                                           
2022-08-15  139.550003  103.32  155.919998  94.739998  90.639999  277.959991   

                  AMZN         AVB         AVY         AXP  ...        PRU  \
Date                                                        ...              
2022-08-15  128.550003  205.080002  180.190002  156.300003  ...  94.080002   

                 PYPL          RE        ROL       ROST         UNH    URI  \
Date                                                                         
2022-08-15  97.660004  280.149994  36.330002  89.459999  509.769989  290.0   

                     V        VRSK        XOM  
Date                                           
2022-08-15  199.410004  185.910004  97.669998  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ       ALLE       AMAT  \
Date                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP         AIZ       ALLE       AMAT  \
Date                                                                   
2022-08-16  142.509995  100.190002  155.759995  93.150002  88.919998   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-08-16  277.350006  126.279999  201.320007  179.360001  156.130005  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-08-16  92.540001  96.400002  285.019989  35.799999  89.559998   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-08-16  522.909973  288.709991  195.369995  183.979996  94.830002  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP         AIZ        ALLE        AMA

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-07-14,0.144629,0.104779,0.102867,0.126579,0.229882,0.262141,0.124662,0.069538,0.246278,0.116345,...,0.243127,0.202516,0.128169,0.121274,0.124628,0.072626,0.110261,0.261637,0.269787,0.265606
2022-07-15,0.144515,0.134341,0.247989,0.276335,0.223340,0.103051,0.146337,0.069538,0.265109,0.116345,...,0.066848,0.202254,0.121900,0.242075,0.239647,0.072626,0.110261,0.070731,0.236140,0.265606
2022-07-18,0.088396,0.104779,0.241526,0.137259,0.229882,0.103051,0.202461,0.256224,0.112971,0.115603,...,0.066848,0.142966,0.236088,0.121274,0.127285,0.104278,0.229480,0.108546,0.063664,0.171711
2022-07-19,0.139838,0.134341,0.119684,0.124390,0.199636,0.242739,0.106146,0.241141,0.244705,0.127756,...,0.123891,0.200278,0.236088,0.083542,0.221498,0.136131,0.214300,0.070731,0.063664,0.171711
2022-07-20,0.199894,0.217724,0.107507,0.230333,0.209332,0.130252,0.202042,0.071275,0.063623,0.108261,...,0.111045,0.142407,0.113645,0.120589,0.090277,0.219212,0.130741,0.070731,0.110016,0.109888
2022-07-21,0.112595,0.107284,0.269278,0.222576,0.209332,0.130252,0.202461,0.070556,0.118756,0.108261,...,0.111045,0.142407,0.128169,0.240608,0.224102,0.219212,0.125746,0.070731,0.124226,0.184110
2022-07-22,0.137874,0.078573,0.068980,0.109664,0.248773,0.103051,0.237715,0.242576,0.064418,0.127140,...,0.259437,0.136938,0.113645,0.232761,0.126890,0.273741,0.135381,0.108546,0.248868,0.184110
2022-07-25,0.274121,0.134341,0.106733,0.111822,0.215096,0.068306,0.105781,0.255512,0.112229,0.269855,...,0.066848,0.201321,0.113645,0.232761,0.241805,0.072626,0.109519,0.110275,0.113932,0.109888
2022-07-26,0.088396,0.137351,0.119997,0.070110,0.214361,0.103051,0.253273,0.117558,0.063742,0.127140,...,0.123891,0.285836,0.065247,0.232761,0.124246,0.139326,0.225278,0.070573,0.236140,0.161609


In [47]:
print(RPS2.mean(axis=1).mean())

0.1581573410209053


In [48]:
print(RPS2.mean(axis=1))

date
2022-07-14    0.168341
2022-07-15    0.163934
2022-07-18    0.146948
2022-07-19    0.158429
2022-07-20    0.144603
2022-07-21    0.147258
2022-07-22    0.161320
2022-07-25    0.162720
2022-07-26    0.166235
2022-07-27    0.148748
2022-07-28    0.157813
2022-07-29    0.160800
2022-08-01    0.163878
2022-08-02    0.152666
2022-08-03    0.156231
2022-08-04    0.158716
2022-08-05    0.163690
2022-08-08    0.160773
2022-08-09    0.160959
2022-08-10    0.161903
2022-08-11    0.146043
2022-08-12    0.161996
2022-08-15    0.161417
2022-08-16    0.160358
dtype: float64


In [50]:
#save results 
RPS2.to_excel('24_July_19_August-grad_boost(with_feature_imp)-one_m_less.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('24_July_19_August-PROBABILITIES-grad_boost(with_feature_imp)-one_m_less.xlsx')